In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd "/content/drive/My Drive/Colab Notebooks/brism/"
!pip install kora -q
from kora import drive
drive.link_nbs()

Mounted at /content/drive
/content/drive/My Drive/Colab Notebooks/brism
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 KB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 25.8 MB/s eta 0:00:00


In [ ]:
%ls

 7.4_QA/
'Beijing urged to weigh environmental impact of belt and road projects.rtf'
'BI detector.ipynb'
 BI_direct_exploration.ipynb
 BI_direct_topic_clustering.ipynb
 data/
 data_scraping/
 ExtractKeywords.ipynb
 GoogleNews-vectors.bin
 images/
 IndividualOrganizationdetector/
 NexisNews.ipynb
 relation_extraction_BiGRU_ARTT/
 SilkRoadProcessing.ipynb
 SimCSE/
 TextNormalizer.ipynb
 TextSimilarity.ipynb
 Twitter_Belt_and_Road.ipynb
 TwitterDataAnalysis1.ipynb
 TwitterDataAnalysis2.ipynb
'Twitter data exploration.ipynb'
 UniversityWebsiteNews.ipynb
 UniWebNewsCNClustering.ipynb
'Untitled document.gdoc'
 WebNewsDataAnalysis.ipynb
 WeiboDataAnalysis.ipynb
 weibo_reddit.ipynb


In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 37.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 81.2 MB/s eta 0:00:00


In [3]:
!pip install python-Levenshtein
!pip install --upgrade scipy networkx


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.0/174.0 KB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 34.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 46.7 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.7.3
    Uninstalling scipy-1.7.3:
      Successfully uninstalled scipy-1.7.3


In [4]:
from Levenshtein import distance as lev
import transformers
import torch
import numpy as np
import networkx as nx
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from sklearn.preprocessing import  MinMaxScaler

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# normalized = './data/twitter_data/normalized_bri.zip'

In [ ]:
# normalized_data  =pd.read_csv(normalized,compression='zip',lineterminator='\n')

In [ ]:
# stopwords_path = './data/stopwords_cn_en.txt'
# with open(stopwords_path,'r') as f:
#     stopwords = f.readlines()
# stopwords = [i.replace('\n', '') for i in stopwords]

In [5]:
import json
from fastcore.imports import NoneType
import pandas as pd
from transformers import BertModel, BertTokenizer
from transformers import AutoModel, AutoTokenizer
import nltk
from nltk.corpus import stopwords
sea_bri_themes = './data/twitter_data/sea_bri_themes.zip'
# stopwords_path = './data/stopwords_cn_en.txt'
bert_path = './data/Bert/'
simcse_path = './SimCSE/result/unsup-simcse-bert-base-uncased/'
theme_cols = ['Admission',
       'Chinese Language', 'Cultural Exchange', 'Economy', 'Education',
       'Employment', 'Immigration', 'Indonesian Language', 'Infrastructure',
       'Institutional Cooperation', 'MOU', 'Malay Language', 'Policy',
       'Research', 'Scholarship', 'Student', 'Study', 'Thai Language',
       'Training', 'University']
save_path = './data/twitter_data/'

In [6]:
class NumpyEncoder(json.JSONEncoder):
    """ Special json encoder for numpy types """
    def default(self, obj):
        if isinstance(obj, (np.int_, np.intc, np.intp, np.int8,
                            np.int16, np.int32, np.int64, np.uint8,
                            np.uint16, np.uint32, np.uint64)):
            return int(obj)
        elif isinstance(obj, (np.float_, np.float16, np.float32,
                              np.float64)):
            return float(obj)
        elif isinstance(obj, (np.ndarray,)):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)

In [ ]:
# encoder_model = AutoTokenizer.from_pretrained(simcse_path)

In [7]:

import math
class TextSimilarity(object):
    def __init__(self, file_path =None,bert_path = None,simcse_path=None,theme_cols = None, save_path= None):
        if file_path:
          self.data = pd.read_csv(file_path,compression='zip',lineterminator='\n')
          self.data = self.data.where(pd.notnull(self.data),None)
        if bert_path:
          self.encoder_model = BertModel.from_pretrained('bert-base-uncased')
          self.encoder_tokenizer = BertTokenizer.from_pretrained(bert_path)
        if simcse_path:
          self.encoder_model = AutoTokenizer.from_pretrained(simcse_path)
          self.encoder_tokenizer = AutoTokenizer.from_pretrained(simcse_path)
        self.theme_cols = theme_cols
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        if save_path:
          self.save_path = save_path
        

    def subset_theme(self,theme_col, text_col):
      # Combine sentences into one single document
      #Remove duplicates in associated_tweets (retweeted tweets) but preserve null values
      # data = self.data[self.data['retweeted'].isnull() | ~self.data[self.data['retweeted'].notnull()].duplicated(subset='retweeted',keep='first')]
      subset = self.data[(self.data[theme_col].isnull()==False) ][[text_col, 'id']]
      texts = [[i for i in t.split() if i not in stopwords.words('english')] for t in subset[text_col]]
      subset['no_stops'] = [' '.join(text) for text in texts]
      subset = subset.drop_duplicates('no_stops') #remove duplicated texts
      dis_matrix = self._edit_distance(subset, 'no_stops') #remove very similar texts based on the sum of word-level edit distance
      np.fill_diagonal(dis_matrix, 10000)
      index_0s = list(set([tuple(set(i)) for i in np.argwhere(dis_matrix <=3)])) #find locations where values <=3
      subset['index'] = range(0, len(subset))
      remove_index = [j[1] for j in index_0s] #retain only column index
      subset = subset[~subset['index'].isin(remove_index)]
      # subset = subset.drop(index=remove_index)

      return subset
    @staticmethod
    def _edit_distance(subset,text_col):
      dis_matrix = np.zeros((len(subset),len(subset)))
      for ida, sent1 in enumerate(subset[text_col].to_list()):
        for idb, sent2 in enumerate(subset[text_col].to_list()):
          dis_matrix[ida][idb] = sum([lev(s1,s2) for s1,s2 in zip(sent1.split(), sent2.split())])
    

      return dis_matrix

        
    def sent_vec(self, subset,text_col,max_length=512,batch_size = 64):
      # self.bert_model.to(self.device)
      #find longest text
      all_input_ids, all_att_mask= [],[]
      texts = subset[text_col].to_list()
      print('Number of texts: {}'.format(len(texts)))
      longest_len = max(subset[text_col].map(lambda x: len(x.split())).to_list())
      print('Length of longest text: {}'.format(longest_len))
      if longest_len >= max_length:
        longest_len = max_length
      print('Encoding text')
      self.sent_vec_dict = {}
      for batch_idx in range(math.ceil(len(texts)/batch_size)):
        batch_input_ids, batch_att_mask= [],[]
      # for num, text in enumerate(texts):      
        for text in texts[batch_size*batch_idx:batch_size*(batch_idx+1)]:
          
          encoded_dict = self.encoder_tokenizer.encode_plus(text, max_length=longest_len,padding = 'max_length',
                                                          return_tensors = 'pt', truncation=True)
          batch_input_ids.append(encoded_dict['input_ids'])
          batch_att_mask.append(encoded_dict['attention_mask'])
        if len(texts) > 700:
          self.dict_path = './data/twitter_data/sent_embedding_dict.txt'
          batch_input_ids = torch.cat(batch_input_ids)
          batch_att_mask = torch.cat(batch_att_mask)
          
          output = self.encoder_model(input_ids = batch_input_ids,
                                      attention_mask = batch_att_mask)
          pooled_output = output['pooler_output'].detach().numpy()
          
          tweet_ids = subset['id'].to_list()[batch_size*batch_idx:batch_size*(batch_idx+1)]
          try:
            existing_dict = {}
            with open(summarizer.dict_path) as f:
                content = f.readlines()
                for c in content:
                    dic = json.loads(c)
                    existing_dict.update(dic)
          except:
            existing_dict = {}
          for id, sent_embedding in zip(tweet_ids,pooled_output):
              if id not in self.sent_vec_dict and id not in existing_dict:
                self.sent_vec_dict[id] = sent_embedding
                print('Getting bert sentence embeddings for tweet {}'.format(id))
          print("Total number of sentences is large. Saving sentence embeddings dictionary")
          
          dumped = json.dumps(self.sent_vec_dict, cls=NumpyEncoder)
          with open(self.dict_path, 'a') as f:
              f.write(dumped + '\n')
              
          
        else:
          all_input_ids += batch_input_ids
          all_att_mask += batch_att_mask
        
      if len(texts)>700:
        return (tweet_ids,)+ (None,)
      else:
        all_input_ids = torch.cat(all_input_ids)
        all_att_mask = torch.cat(all_att_mask)
        print('Getting bert sentence embeddings')
        output = self.encoder_model(input_ids = all_input_ids,
                                    attention_mask = all_att_mask)
        pooled_output = output['pooler_output'].detach().numpy()

        return (None,)+ (pooled_output,)
      
      

    # calculate textrank
    def _calc_text_rank(self,sentence_vec=None, tweet_ids=None):
      #sentence as node, add an edge when cosine similarity is above threshold, cosine similarity as weight
      #pagerank scores give ranks of sentences based on node weights + cosine similiary 
      # sentence_vec：[]
      if tweet_ids:
        with open(self.dict_path) as json_file:
            sent_vec_dic = json.load(json_file)
            sentence_vec = [np.array(sent_vec_dic[id]) for id in tweet_ids]
            assert len(sentence_vec) !=0, 'Something wrong with sentence embeddings'
      sim_mat = cosine_similarity(sentence_vec) # 只有x，计算x所有sample的笛卡尔积， 
      np.fill_diagonal(sim_mat, 0) # 对角线置0
      nx_graph = nx.from_numpy_array(sim_mat)
      tol, max_iter = 1e-7, 1000
      Flag = True
      while Flag:
          try:
              pagerank_score = nx.pagerank(nx_graph, tol=tol, max_iter=max_iter)
              Flag = False
          except nx.PowerIterationFailedConvergence as e:
              print(e)
              tol *= 10
      pagerank_score = np.array([v for k, v in sorted(pagerank_score.items(), key=lambda x: x[0])])
      return pagerank_score
    
    # 执行MMR算法
    # 贪心算法：每次计算每个句子的mmr_score，选择最高分
    def MMR(self,subset, extract_num,pagerank_score,sentence_vec,alpha = 0.8, save_file_name=None):
      n = extract_num
      summary_set = []
      max_score_index = np.argmax(pagerank_score) # 起始,先选择最大的一个
      summary_set.append(max_score_index) 
      print('sentence_vec shape:',sentence_vec.shape)
      while n > 0:
          # 冗余性
          sim_mat = cosine_similarity(sentence_vec, sentence_vec[summary_set]) 
          sim_mat = np.max(sim_mat,axis=1)  # choose the biggest value by row -> 1d array: 
          #recalculate similarity score between orginal sentence and sentences in subset and return the most similar ones from subset
          
          # 两个score数据对齐
          scaler = MinMaxScaler() 
          feature_score = np.array([pagerank_score, sim_mat]).T 
          scaler.fit(feature_score)
          feature_score = scaler.transform(feature_score)
          [pagerank_score, sim_mat] = feature_score[:,0], feature_score[:,1] 

          # 计算mmr（减去冗余性）
          mmr_score =  alpha*pagerank_score - (1-alpha)*sim_mat

          mmr_score[summary_set] = -100 # 将已经是摘要的mask掉
          max_index  = np.argmax(mmr_score) # 取出来最高得分
          summary_set.append(max_index)
          n -= 1  # 控制摘要数量
      summary_subset = subset[subset['index'].isin(summary_set)]
      if save_file_name:
        summary_subset.to_csv(save_path+'{}.csv'.format(save_file_name)
                              )
        print('{}.csv saved!'.format(save_file_name))
      return summary_subset

        

In [ ]:
# summarizer = TextSimilarity(file_path=sea_bri_themes, bert_path = bert_path,save_path = save_path)

<ipython-input-111-d49da852fbc0>:1: DtypeWarning: Columns (15,36,41) have mixed types.Specify dtype option on import or set low_memory=False.
  summarizer = TextSimilarity(file_path=sea_bri_themes, bert_path = bert_path,save_path = save_path)
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initiali

In [ ]:
# theme = 'Economy'
# text_col = 'lowered_norm_text'
# ratio = 0.5
# theme_subset = summarizer.subset_theme(theme, text_col)
# extract_num = len(theme_subset)*ratio if len(theme_subset) > 50 else len(theme_subset)
# outputs= summarizer.sent_vec(theme_subset, text_col)

# Running summarization

In [8]:
class SummThemeSubset(object):
  def __init__(self):
    self.summarizer = TextSimilarity(file_path=sea_bri_themes, bert_path = bert_path,save_path = save_path)
  def run(self, theme,text_col='lowered_norm_text',save_file_name=None, ratio = 0.3):
    theme_subset = self.summarizer.subset_theme(theme, text_col)
    extract_num = len(theme_subset)*ratio if len(theme_subset) > 50 else len(theme_subset)
    print('Getting sentence embeddings')
    tweet_ids,sent_vectors= self.summarizer.sent_vec(theme_subset, text_col)
    print('Calculating text rank')

    if tweet_ids == None:
      pagerank_score = self.summarizer._calc_text_rank(sentence_vec=sent_vectors, tweet_ids=tweet_ids)
    else:
      sent_vec_dit = {}
      with open(self.summarizer.dict_path) as f:
          content = f.readlines()
          for c in content:
              dic = json.loads(c)
              sent_vec_dit.update(dic)
          sent_vectors = np.array(list(sent_vec_dit.values()))
          sent_vectors =  np.reshape(sent_vectors,(-1,768))

          print('Number of sentence with embeddings:',len(sent_vectors))
      pagerank_score = self.summarizer._calc_text_rank(sentence_vec = sent_vectors)
    print(extract_num)
    summ_theme_subset = self.summarizer.MMR(theme_subset,extract_num,pagerank_score,sent_vectors,save_file_name=save_file_name)
    return summ_theme_subset

In [9]:
themeSumm = SummThemeSubset()

<ipython-input-8-df25a7f1a6da>:3: DtypeWarning: Columns (15,36,41) have mixed types.Specify dtype option on import or set low_memory=False.
  self.summarizer = TextSimilarity(file_path=sea_bri_themes, bert_path = bert_path,save_path = save_path)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [20]:
# Student_summary = themeSumm.run('Student',text_col='lowered_norm_text',save_file_name='summary_Student',ratio = 0.5)
# Admission_summary = themeSumm.run('Admission',text_col='lowered_norm_text',save_file_name='summary_Admission',ratio = 0.5)
# ChineseLanguage_summary = themeSumm.run('Chinese Language',text_col='lowered_norm_text',save_file_name='summary_ChineseLanguage',ratio = 0.5)
# CulturalExchange_summary = themeSumm.run('Cultural Exchange',text_col='lowered_norm_text',save_file_name='summary_CulturalExchange',ratio = 0.5)
# Economy_summary = themeSumm.run('Economy',text_col='lowered_norm_text',save_file_name='summary_Economy',ratio = 0.5)
# Education_summary = themeSumm.run('Education',text_col='lowered_norm_text',save_file_name='summary_Education',ratio = 0.5)
# Employment_summary = themeSumm.run('Employment',text_col='lowered_norm_text',save_file_name='summary_Employment',ratio = 0.5)
# Immigration_summary = themeSumm.run('Immigration',text_col='lowered_norm_text',save_file_name='summary_Immigration',ratio = 0.5)
# IndonesianLanguage_summary = themeSumm.run('Indonesian Language',text_col='lowered_norm_text',save_file_name='summary_IndonesianLanguage',ratio = 0.5)
# Infrastructure_summary = themeSumm.run('Infrastructure',text_col='lowered_norm_text',save_file_name='summary_Infrastructure',ratio = 0.5)
# InstitutionalCooperation_summary = themeSumm.run('Institutional Cooperation',text_col='lowered_norm_text',save_file_name='summary_InstitutionalCooperation',ratio = 0.5)
# MalayLanguage_summary = themeSumm.run('Malay Language',text_col='lowered_norm_text',save_file_name='summary_MalayLanguage',ratio = 0.5)
# MOU_summary = themeSumm.run('MOU',text_col='lowered_norm_text',save_file_name='summary_MOU',ratio = 0.5)
# Policy_summary = themeSumm.run('Policy',text_col='lowered_norm_text',save_file_name='summary_Policy_summary',ratio = 0.5)
# Scholarship_summary = themeSumm.run('Scholarship',text_col='lowered_norm_text',save_file_name='summary_Scholarship',ratio = 0.5)
# Research_summary = themeSumm.run('Research',text_col='lowered_norm_text',save_file_name='summary_Research',ratio = 0.5)
# Study_summary = themeSumm.run('Study',text_col='lowered_norm_text',save_file_name='summary_Study',ratio = 0.5)
# ThaiLanguage_summary = themeSumm.run('Thai Language',text_col='lowered_norm_text',save_file_name='summary_ThaiLanguage',ratio = 0.5)
# Training_summary = themeSumm.run('Training',text_col='lowered_norm_text',save_file_name='summary_Training',ratio = 0.5)
University_summary = themeSumm.run('University',text_col='lowered_norm_text',save_file_name='summary_University',ratio = 0.5)




Getting sentence embeddings
Number of texts: 251
Length of longest text: 48
Encoding text
Getting bert sentence embeddings
Calculating text rank
125.5
sentence_vec shape: (251, 768)
summary_University.csv saved!


# Comparing stemming and lemmatizing methods

In [ ]:

# WORDNET LEMMATIZER (with appropriate pos tags)
 
import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
from nltk.corpus import wordnet
 
lemmatizer = WordNetLemmatizer()
 
# Define function to lemmatize each word with its POS tag
 
# POS_TAGGER_FUNCTION : TYPE 1
def pos_tagger(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:         
        return None

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


### WordNetLemmatizer based on pos tags

In [ ]:
text = 'zust silk road scholarship for undergraduate studies to finance outstanding students from countries within the belt and road initiative to pursue studies in zhejiang university of science and technology zust we offer silk road chinese'

In [ ]:
pos_tagged = nltk.pos_tag(nltk.word_tokenize(text)) 
wordnet_tagged = list(map(lambda x: (x[0], pos_tagger(x[1])), pos_tagged))
lemmatized_sentence = []
for word, tag in wordnet_tagged:
    if tag is None:
        # if there is no available tag, append the token as is
        lemmatized_sentence.append(word)
    else:       
        # else use the tag to lemmatize the token
        lemmatized_sentence.append(lemmatizer.lemmatize(word, tag))
lemmatized_sentence = " ".join(lemmatized_sentence)

In [ ]:
lemmatized_sentence

'zust silk road scholarship for undergraduate study to finance outstanding student from country within the belt and road initiative to pursue study in zhejiang university of science and technology zust we offer silk road chinese'

### SnowballStemmer

In [ ]:

import nltk
nltk.download('punkt')
from nltk.stem.snowball import SnowballStemmer

snowBallStemmer = SnowballStemmer("english")
text = 'zust silk road scholarship for economic studies to finance outstanding students from countries within the belt and road initiative to pursue studies in zhejiang university of science and technology zust we offer silk road chinese'
wordList = nltk.word_tokenize(text)

stemWords = [snowBallStemmer.stem(word) for word in wordList]

print(' '.join(stemWords))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


zust silk road scholarship for econom studi to financ outstand student from countri within the belt and road initi to pursu studi in zhejiang univers of scienc and technolog zust we offer silk road chines


In [ ]:
snowBallStemmer.stem('economic studies to finance')

'economic studies to fin'

### krovetzstemmer

In [ ]:
!pip install krovetzstemmer
from krovetzstemmer import Stemmer 
stemmer = Stemmer() 
text = 'zust silk road scholarship for undergraduate studies to finance outstanding students from countries within the belt and road initiative to pursue studies in zhejiang university of science and technology zust we offer silk road chinese'
wordList = nltk.word_tokenize(lemmatized_sentence)
stemWords = [stemmer.stem(word) for word in wordList] # got: ‘utility’ stemmer.stem(u’utilities’)
print(' '.join(stemWords))

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
zust silk road scholarship for undergraduate study to finance outstanding student from country within the belt and road initiative to pursue study in zhejiang university of science and technology zust we offer silk road china


international conference on the belt and road and macao development coming up fdlx business news economy finance policy


In [ ]:
# !pip install spacy
# import spacy
# nlp = spacy.load("en_core_web_sm")
# text = 'international conference on the belt and road and macao development coming up fdlx business news economy finance policy'
# doc = nlp(text)

# for token in doc:
#     print(token,token.lemma_)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
summarizer.data[summarizer.data.text.str.contains('financ')][['stemmed','lowered_norm_text']]

,stemmed,lowered_norm_text
10,"['bri', 'the', 'new', 'silk', 'road', 'is', 'b...",bri the new silk road is being managed and wel...
11,"['beij', 'multibillion', 'dollar', 'initi', 's...",beijing multibillion dollar initiative seen as...
67,"['britain', 'say', 'it', 'natur', 'partner', '...",britain says it natural partner for china new ...
68,"['britain', 'say', 'it', 'natur', 'partner', '...",britain says it natural partner for china new ...
304,"['german', 'zypri', 'to', 'visit', 'beij', 'fo...",german zypries to visit beijing for new silk r...
...,...,...
22248,"['in', 'beij', 'moreno', 'push', 'trade', 'loa...",in beijing moreno pushes trade loan refinancin...
22878,"['jin', 'qi', 'chairman', 'of', 'silk', 'road'...",jin qi chairman of silk road fund addressed sp...
23496,"['wari', 'of', 'debt', 'trap', 'govern', 'reth...",wary of debt trap government rethinks silk roa...
23595,"['both', 'venic', 'and', 'singapor', 'have', '...",both venice and singapore have the lion as the...


# Combine summaries into txt file for SimSCE training

In [21]:
import glob
import os
def concat(csv_files):
    all_ = pd.DataFrame()
    for i in range(len(csv_files)):
        try:
            df = pd.read_csv(csv_files[i],encoding="utf-8")
            df['keyword'] = csv_files[i].split('/')[-1].split('.')[0].split('_')[-1]
            all_ = all_.append(df)
        except:
            continue
    return all_
csv_files = glob.glob('./data/twitter_data/summary_*.csv')

In [22]:
summary_themes = concat(csv_files)
summary_themes.to_csv('./data/twitter_data/summary_themes.csv')
with open('./data/twitter_data/summary_themes.txt', 'w') as f:
  for i in summary_themes.lowered_norm_text:
    f.write('%s\n' % i)

# Text augmentation

### Textaugment

In [23]:
!pip install textaugment
!pip install nlpcda
!pip install nlpaug

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 KB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 KB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 KB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 KB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 KB 7.5 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-3.0.0-py3-none-any.whl size=15735 sha256=0409573d7720677a0637c6d11a372c3d895d1e45609fbdf542405bc2126f1def
  Stored in directory: /root/.cache/pip/wheels/0e/ce/9b/d51de1064911d42480ab6b57fc943ee36572441f27546354e2
Successfully built googletrans
  Attempting uninstall: chardet
    Found existing installation: chardet 4.0.0
   

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.2/527.2 KB 11.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for nlpcda: filename=nlpcda-2.5.8-py3-none-any.whl size=527645 sha256=96d1f79a37faf5a42af83676d95ad10e9e1554dd5241fe118251ff4a8aafc823
  Stored in directory: /root/.cache/pip/wheels/39/65/43/72953fd062c5b4400bee494df0779db3f935f945c78a2e5b73
Successfully built nlpcda
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.5/410.5 KB 8.9 MB/s eta 0:00:00


In [75]:
file_path = './data/twitter_data/summary_themes.csv'
import nlpaug.flow as naf
import nlpaug.augmenter.char as nac
import nlpaug.augmenter.word as naw
import random
class TextAugmentation(object):
  def __init__(self, df=None, file_path =None):
      if df is not None:
        self.df= df
      else:
        if file_path.split('.')[-1] == 'zip':
          self.df = pd.read_csv(file_path,compression='zip',lineterminator='\n')
        if file_path.split('.')[-1] == 'csv':
          self.df = pd.read_csv(file_path)
    
    # self.naf = naf
    # self.nac = nac
    # self.naw = naw

  def textaugmenter(self,runs=1,v=False ,n=True, p=0.3):
    
      import nltk
      nltk.download('punkt')
      nltk.download('wordnet')
      nltk.download('averaged_perceptron_tagger')
      nltk.download('omw-1.4')
      from textaugment import Wordnet
      from textaugment import Word2vec
      import gensim
      model = gensim.models.KeyedVectors.load_word2vec_format('./GoogleNews-vectors.bin', binary=True)
      t = Wordnet(v=v ,n=n, p=p)
      t_2 = Word2vec(model=model, runs=runs, v=v, p=p)
      
      # augmented = []
      # for text in self.df[col]:
      #   augmented.apppend(t.augment(text))
      
      # augmented_2 = []
      # for text in augmented:
      #   augmented_2.append(t_2.augment(text))
      # assert len(augmented_2) == len(self.df), "The number of augmented texts is not equal to original texts!"
      return t,t_2
  @staticmethod
  def create_negative_pairs(theme_cols):
    themes = [''.join(col.split()) for col in theme_cols]
    conflicting_themes = ['University','Scholarship','Student','Study']
    theme_negatives = {}
    for i, theme in enumerate(themes):
      if theme not in conflicting_themes:
        theme_negatives[theme] = themes[:i]+themes[i+1:]
      else:
        candicates = themes[:i]+themes[i+1:] 
        theme_negatives[theme] = [c for c in candicates if c not in conflicting_themes]
    return theme_negatives

  @staticmethod
  def config_augmenter(model_name, aug_min=1, aug_max=10, aug_p=0.3, lang='eng',stopwords=None, tokenizer=None, action='delete'):
      if model_name == 'SynonymAug':
        aug = naw.SynonymAug(aug_src='wordnet', model_path=None, name='Synonym_Aug', aug_min=aug_min, aug_max=aug_max, aug_p=aug_p, lang=lang,
                            stopwords=stopwords, tokenizer=tokenizer, reverse_tokenizer=None, stopwords_regex=None, force_reload=False, 
                            verbose=0)
      if model_name == 'AntonymAug':
        aug = naw.AntonymAug(name='Antonym_Aug', aug_min=aug_min, aug_max=aug_max, aug_p=aug_p, lang=lang, stopwords=stopwords, tokenizer=tokenizer, 
                        reverse_tokenizer=None, stopwords_regex=None, verbose=0)
      if model_name == 'RandomWordAug':
        aug = naw.RandomWordAug(action=action, name='RandomWord_Aug',  aug_max=aug_max, aug_p=aug_p, stopwords=stopwords,  # ‘substitute’, ‘swap’, ‘delete’ or ‘crop’.
                          target_words=None, tokenizer=tokenizer, reverse_tokenizer=None, stopwords_regex=None, verbose=0)
      if model_name=='ContextualWordEmbsAug':
        aug = naw.ContextualWordEmbsAug(model_path='bert-base-uncased', model_type='', action=action, top_k=100, name='ContextualWordEmbs_Aug', #‘substitute’/ insert
                                        aug_min=aug_min, aug_max=aug_max, aug_p=aug_p, stopwords=None,
                                        batch_size=32, device='cpu', force_reload=False, stopwords_regex=None, verbose=0, silence=True, use_custom_api=True)
      return aug
  def create_pos_neg_samples(self,col, theme_cols, model1=None,model2=None,model3=None,aug_p=0.3):
      self.df = self.df[self.df.keyword.isin([''.join(col.split()) for col in theme_cols])]
      theme_negatives_dict = self.create_negative_pairs(theme_cols)
      #Augmenter 1: from nlpaug
      if model1 and model2 and model3:
        flow = naf.Sometimes([model1, model2,model3],aug_p=aug_p)
      if model1 and model2:
        flow = naf.Sometimes([model1, model2],aug_p=aug_p)
      
 
      #Augmenter 1: from textaugment
      text_augmenter1, text_augmenter2 = self.textaugmenter()

      augemented = []
      negatives = []

      for text,theme in zip(self.df[col], self.df['keyword']):
        print('Generating positive/augmented samples for the text')
        flow_augmented= flow.augment(text)[0]
        text_augmented = text_augmenter1.augment(text)
        text_augmented = text_augmenter2.augment(text_augmented)
        print('original_text:', text)
        print('flow augmented:', flow_augmented)
        print('text augmented:', text_augmented)
        augemented.append((flow_augmented,text_augmented))

        print('Generating negative samples for the text')
        candidates = theme_negatives_dict[theme]
        candidates_list = self.df[augumenter.df.keyword.isin(candidates)].lowered_norm_text.to_list()
        candidates_samples =random.sample(candidates_list, len(candidates_list))
        negative_sample1 = random.choice(candidates_samples)
        negative_sample2 = random.choice(candidates_samples)
        print('negative sample1 :', negative_sample1)
        print('negative sample2 :', negative_sample2)
        negatives.append((negative_sample1,negative_sample2))
    

      len(augemented) == len(self.df), "The number of augmented texts is not equal to original texts!"
      return augemented,negatives










    
    

In [76]:
augumenter = TextAugmentation(file_path =file_path)

In [33]:
# augumenter.df.keyword.unique()

array(['Policy', 'themes', 'University', 'Training', 'Study', 'Research',
       'Scholarship', 'summary', 'MOU', 'InstitutionalCooperation',
       'Admission', 'Student', 'ChineseLanguage', 'Education',
       'Employment', 'Immigration', 'Infrastructure', 'Economy'],
      dtype=object)

In [50]:
aug1 = augumenter.config_augmenter('ContextualWordEmbsAug',aug_p=0.4,action = 'substitute')
aug2 = augumenter.config_augmenter('RandomWordAug',aug_p=0.4, action = 'delete')
aug3 = augumenter.config_augmenter('SynonymAug',aug_p=0.4)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [78]:
augemented,negatives = augumenter.create_pos_neg_samples('lowered_norm_text',theme_cols, model1= aug1, model2=aug2,model3=aug3,aug_p=0.3)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Streaming output truncated to the last 5000 lines.
negative sample1 : china economic growth is building new silk road by bringing business back to its ancient capital learn more when you visit china amp singapore on the new sit malaysia program
negative sample2 : silk road heritage crowdsourcing project phd scholarship at university of nottingham ningbo china youth opportunities
Generating positive/augmented samples for the text
original_text: the chinese foreign ministry said beijing ambitious belt and road global trade infrastructure initiative will be on the agenda as the two countries look to improve their economic ties
flow augmented: ['the chinese press secretary declared several ambitious belt and road global governance infrastructure reforms would be on the pace after the two nations look to improve their economic ties']
text augmented: the chinese foreign ministry siad thailand ambitious belt and paths gloabl trade lambert_martelle initiative will remain through which docket a

In [ ]:
df = pd.DataFrame(columns = ['text1','text2', 'label'])

for num, (pos, neg, text) in enumerate(zip(augemented, negatives, augumenter.df.lowered_norm_text.to_list())):
  
  # print([neg[0]]+[neg[1]])
  # break
  # 
  print('No.{}'.format(num))
  text1 = [text]*4
  text2 = pos[0]+[pos[1]]+[neg[0]]+[neg[1]]
  label = [1]*2+[0]*2
  data = pd.DataFrame(list(zip(text1,text2,label)), columns = ['text1','text2', 'label'])
  df = pd.concat([df, data])






In [94]:
df.to_csv('./data/twitter_data/cross_bert_samples.csv')